# biLSTM: Automatic glossing, otomi (corpus completo)

In [30]:
from reccurrent_model import biLSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np
import torch

Sólo toma información de caracter y etiqueta. Elimina POS tags.

In [2]:
file1 = load(open('pickle_objects/predata','rb'))
print(file1[0], len(file1))

#file2 = load(open('pickle_objects/preinput_data','rb'))
#print(file2[0], len(file2))

#file3 = load(open('pickle_objects/X_input','rb'))
#print(file3[0][:8], len(file3))

#file4 = load(open('pickle_objects/y_input','rb'))
#print(file4[0], len(file4))

[[['n', 'v', 'B-psd'], ['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['μ', 'v', 'I-stem'], ['d', 'v', 'I-stem'], ['i', 'v', 'I-stem']], [['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['ε', 'v', 'I-stem'], ['p', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['í', 'v', 'I-stem']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['t', 'v', 'B-lig'], ["'", 'v', 'B-stem'], ['μ', 'v', 'I-stem'], ['n', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-stem'], ['ä', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['r', 'v', 'I-stem'], ['a', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['á', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['k', 'obl', 'B-stem'], ['o', 'obl', 'I-stem']], [['c', 'obl', 'B-stem'], ['h', 'obl', 'I-stem'], ['í', 'obl', 'I-stem'], ['k', 'obl', 'I-stem'], ['ó', 'obl', 'I-stem'], ['h', 'obl', 'I-stem'], ['t', 'obl', 'I-

In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))
    
#print(data)

### Training and test data

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_sents, train_tags = zip(*train_pairs)

#print(train_sents)
#print(train_tags)

1196 590


### Training the model

In [5]:
model = biLSTM(train_sents, train_tags)

In [6]:
%%time
model.train_model(its=150)

100%|██████████| 150/150 [1:02:41<00:00, 25.07s/it]

CPU times: user 10h 48min 36s, sys: 4min 46s, total: 10h 53min 22s
Wall time: 1h 2min 41s


### Evaluation

In [22]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = model.forward(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [23]:
y_true = list(chain(*y_true))
y_pred = list(chain(*y_pred))

print(accuracy_score(y_true, y_pred))

0.7042671379876279


In [27]:
labels = list(set(y_true))
prec, rec, f1, supp = precision_recall_fscore_support(y_true, y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
I-3.imp,0.0,0.0,0.0,8.0
B-dem,0.0,0.0,0.0,56.0
B-prt,0.0,0.0,0.0,14.0
B-det,0.0,0.0,0.0,242.0
B-aqui,0.0,0.0,0.0,2.0
...,...,...,...,...
B-2.pss,0.0,0.0,0.0,16.0
B-3.icp.irr,0.0,0.0,0.0,15.0
I-como,0.0,0.0,0.0,1.0
B-1.cpl.irr,0.0,0.0,0.0,5.0


In [29]:
print(results.to_string())

             Precision    Recall        F1  Support
I-3.imp       0.000000  0.000000  0.000000      8.0
B-dem         0.000000  0.000000  0.000000     56.0
B-prt         0.000000  0.000000  0.000000     14.0
B-det         0.000000  0.000000  0.000000    242.0
B-aqui        0.000000  0.000000  0.000000      2.0
B-por.que     0.000000  0.000000  0.000000      1.0
B-it          0.000000  0.000000  0.000000     18.0
I-dual.exc    0.000000  0.000000  0.000000     15.0
B-prag        0.000000  0.000000  0.000000    113.0
I-mientras    0.000000  0.000000  0.000000      3.0
B-para        0.000000  0.000000  0.000000      1.0
B-2.prf       0.000000  0.000000  0.000000      4.0
B-tiempo      0.000000  0.000000  0.000000      1.0
B-pueblo      0.000000  0.000000  0.000000      2.0
I-3.sg        0.000000  0.000000  0.000000      6.0
I-1.icp.irr   0.000000  0.000000  0.000000      6.0
I-neg         0.000000  0.000000  0.000000      8.0
I-1.cnt       0.000000  0.000000  0.000000     38.0
B-dim       

In [32]:
#torch.save(model,'Model.biLSTM.justChar')